In [1]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [2]:
import numpy as np
import keras
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier

In [3]:
import myModel

new_model = myModel.load_model()

In [3]:
import myModelSquezzeDenseWB

new_modelDense = myModelSquezzeDenseWB.load_model()

In [3]:
import myModelRawSqueezeNet

new_modelRaw = myModelRawSqueezeNet.load_model()

In [4]:
new_modelRaw.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
data_unnormalized (InputLayer)  [(None, 227, 227, 3) 0                                            
__________________________________________________________________________________________________
data_ (Normalization)           (None, 227, 227, 3)  309175      data_unnormalized[0][0]          
__________________________________________________________________________________________________
conv1_ (Conv2D)                 (None, 113, 113, 64) 1792        data_[0][0]                      
__________________________________________________________________________________________________
re_lu (ReLU)                    (None, 113, 113, 64) 0           conv1_[0][0]                     
______________________________________________________________________________________________

In [5]:
def sample_detach (imgs_dir, labels_dir, validation = False):
    
    X = np.load(imgs_dir)
    Y = np.load(labels_dir)
    
    if validation:

        #divide os dados em 80% para treino e 20% para teste
        train_X, test_X, train_label, test_label = train_test_split(X, Y, test_size=0.2, random_state=13)

        #divide os dados de treino em 80% para treino e 20% para validação
        train_X, valid_X, train_label, valid_label = train_test_split(train_X, train_label, test_size=0.2, random_state=13)

        #mostra os shapes resultantes das divisões
        print("Training, test and validation shape")
        print("Training:   ", train_X.shape, train_label.shape)
        print("Validation: ", valid_X.shape, valid_label.shape)
        print("Testing:    ", test_X.shape, test_label.shape)
        
        return train_X, train_label, test_X, test_label, valid_X, valid_label
    else:
        
        #divide os dados em 80% para treino e 20% para teste
        train_X, test_X, train_label, test_label = train_test_split(X, Y, test_size=0.2, random_state=13)

        #divide os dados de treino em 80% para treino e 20% para validação
        #train_X, valid_X, train_label, valid_label = train_test_split(train_X, train_label, test_size=0.2, random_state=13)

        #mostra os shapes resultantes das divisões
        print("Training and test shape")
        print("Training:   ", train_X.shape, train_label.shape)
        print("Testing:    ", test_X.shape, test_label.shape)
        
        return train_X, train_label, test_X, test_label

In [6]:
def create_intermediate_model (model, layerName):
    intermediate_layer_model = keras.Model(inputs=model.input,
                                       outputs=model.get_layer(layerName).output)
    return intermediate_layer_model

In [7]:
def predict_rf (model, layerName, train_X, train_label, test_X, test_label, crit = 'gini', numberEst = 100):
    
    intermediate_layer_model = create_intermediate_model(model, layerName)
    
    intermediate_output = intermediate_layer_model.predict(train_X)
    intermediate_output = np.reshape(intermediate_output, (intermediate_output.shape[0], -1))
    
    clf = RandomForestClassifier(n_estimators = numberEst, criterion = crit)
    predicted_classes = np.argmax(train_label, axis=1) #no onehot
    clf.fit(intermediate_output, predicted_classes)
    
    intermediate_output_test = intermediate_layer_model.predict(test_X)
    intermediate_output_test = np.reshape(intermediate_output_test, (intermediate_output_test.shape[0], -1))
    
    y_pred=clf.predict(intermediate_output_test)
    predicted_classes = np.argmax(test_label, axis=1)
    print("Accuracy Random Forest:",metrics.accuracy_score(predicted_classes, y_pred))

In [8]:
train_X, train_label, test_X, test_label = sample_detach('E:/WB_cells_data/all_imgs_227.npy', 'E:/WB_cells_data/onehot.npy')

Training and test shape
Training:    (7965, 227, 227, 3) (7965, 4)
Testing:     (1992, 227, 227, 3) (1992, 4)


In [14]:
predict_rf(new_modelRaw, 'global_average_pooling2d', train_X, train_label, test_X, test_label, crit = 'log_loss')

Accuracy Random Forest: 0.8177710843373494
